In [1]:
using Pkg; Pkg.activate(".")
# Import libraries
using Flux, Plots, Random, Statistics, LaplaceRedux, MLUtils

In [ ]:
using LaplaceRedux.Data
n = 300       # number of observations
σtrue = 0.30  # true observational noise
x, y = Data.toy_data_regression(n;noise=σtrue)
xs = [[x] for x in x]
X = permutedims(x)

dl = DataLoader((x,y), batchsize=5)

In [ ]:
data = zip(xs,y)
n_hidden = 50
D = size(X,1)
nn = Chain(
    Dense(D, n_hidden, tanh),
    Dense(n_hidden, 1)
)  
loss(x, y) = Flux.Losses.mse(nn(x), y)

In [ ]:
using Flux.Optimise: update!, Adam
opt = Adam(1e-3)
epochs = 1000
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
show_every = epochs/10

for epoch = 1:epochs
  for d in data
    gs = gradient(Flux.params(nn)) do
      l = loss(d...)
    end
    update!(opt, Flux.params(nn), gs)
  end
  if epoch % show_every == 0
    println("Epoch " * string(epoch))
    @show avg_loss(data)
  end
end

In [12]:
la = Laplace(nn; likelihood=:regression, hessian_structure=:full, subset_of_weights=:all, backend=:EmpiricalFisher)
fit!(la, data)
la.H